In [1]:
from pydub import AudioSegment
from pydub.utils import get_array_type
from plotnine import *
import numpy as np
import pandas as pd

/Users/sacheendra/anaconda/envs/multimedia_search/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
filePath = "EUDC 2015 Round 6 A.mp3"

In [ ]:
sound = AudioSegment.from_mp3(filePath)
left = sound.split_to_mono()[0]

left.export("test.wav", format="wav")

In [ ]:
def findInterruptions(speech, threshold):
    # Will have tuples of the format (start location in array, length)
    interruptions = []
    isInterruption = False
    interruptionInstance = (0, 0)
    for i in range(0, len(speech)):
        speechAmplitude = speech[i]
        if not isInterruption:
            if abs(speechAmplitude) < threshold:
                isInterruption = True
                interruptionInstance = (i, 0)
            else:
                continue
        else:
            if abs(speechAmplitude) < threshold:
                continue
            else:
                isInterruption = False
                interruptionInstance = (interruptionInstance[0], i - interruptionInstance[0])
                interruptions.append(interruptionInstance)
                interruptionInstance = (0, 0)

In [ ]:
x = findInterruptions(debatesRead[0], 30)

In [6]:
from vad import VoiceActivityDetector

v = VoiceActivityDetector("test.wav")
# v.plot_detected_speech_regions()

In [7]:
detected_windows = v.detect_speech()

vad.py:149: RuntimeWarning: invalid value encountered in double_scalars
  speech_ratio = sum_voice_energy/sum_full_energy


In [8]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [9]:
zero_segments = zero_runs(detected_windows[:, 1])

In [10]:
zero_segment_lengths = zero_segments[:, 1] - zero_segments[:, 0]

In [18]:
gps = pd.DataFrame(zero_segment_lengths, columns=["seglen"]).groupby(by="seglen").seglen.count()

In [22]:
gps[-50:]

seglen
87       2
88       3
89       1
90       3
91       3
92       2
94       6
95       5
96       4
97       3
98       2
99       4
100      4
101      1
102      2
103      1
104      1
105      2
107      1
108      1
109      1
112      1
113      1
114      2
115      1
116      1
117      1
118      1
121      1
124      1
126      1
129      2
132      1
133      1
138      1
146      1
149      1
150      1
151      1
157      1
159      1
170      1
196      1
223      1
239      1
247      1
295      1
302      1
2060     1
99248    1
Name: seglen, dtype: int64